In [1]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [3]:
#read in incidence csv from github

mortality_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Age_Mortality_Cancer_DB.csv")                         
#get rid of commas in columns
mortality_df = mortality_df.replace(',','', regex=True)
mortality_df.head()     

,average_annual_count,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,county_state,index_of_medical_underservice_score,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019
0,4,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,38.8,Marengo County AL,54.0,19766,19525,19396,19056,18863
1,48,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,50+,606.5,Marengo County AL,54.0,19766,19525,19396,19056,18863
2,18,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<65,84.7,Marengo County AL,54.0,19766,19525,19396,19056,18863
3,35,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,65+,963.0,Marengo County AL,54.0,19766,19525,19396,19056,18863
4,9,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,30.2,Jackson County AL,53.0,52195,51988,51828,51621,51626


In [4]:
#check dtypes
mortality_df.dtypes

average_annual_count                     int64
primary_state_name                      object
cancer                                  object
race_ethnicity                          object
sex                                     object
age                                     object
age_adjusted_rate                       object
county_state                            object
index_of_medical_underservice_score    float64
popestimate2015                          int64
popestimate2016                          int64
popestimate2017                          int64
popestimate2018                          int64
popestimate2019                          int64
dtype: object

In [5]:
#get count of rows
mortality_df.count()

average_annual_count                   8170
primary_state_name                     8170
cancer                                 8170
race_ethnicity                         8170
sex                                    8170
age                                    8170
age_adjusted_rate                      8170
county_state                           8170
index_of_medical_underservice_score    7091
popestimate2015                        8170
popestimate2016                        8170
popestimate2017                        8170
popestimate2018                        8170
popestimate2019                        8170
dtype: int64

In [6]:
#get count of values for age
mortality_df["age"].value_counts()

50+    2420
65+    2406
<65    2198
<50    1146
Name: age, dtype: int64

In [7]:
#get count of NA values for MUA
mortality_df["index_of_medical_underservice_score"].isna().sum()

1079

In [8]:
#drop rows where MUA index is null
mortality_df = mortality_df[mortality_df["index_of_medical_underservice_score"].isna() == False]

In [9]:
#get count of rows
mortality_df.count()

average_annual_count                   7091
primary_state_name                     7091
cancer                                 7091
race_ethnicity                         7091
sex                                    7091
age                                    7091
age_adjusted_rate                      7091
county_state                           7091
index_of_medical_underservice_score    7091
popestimate2015                        7091
popestimate2016                        7091
popestimate2017                        7091
popestimate2018                        7091
popestimate2019                        7091
dtype: int64

In [10]:
#create target 
y = mortality_df[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
0,38.8
1,606.5
2,84.7
3,963.0
4,30.2


In [11]:
#create feautures variable holding ages a d mua score
X = mortality_df[["age", "index_of_medical_underservice_score"]]
X.head()

,age,index_of_medical_underservice_score
0,<50,54.0
1,50+,54.0
2,<65,54.0
3,65+,54.0
4,<50,53.0


In [12]:
#create dummy variables for age
X = pd.get_dummies(X, columns=["age"])
X.head()

,index_of_medical_underservice_score,age_50+,age_65+,age_<50,age_<65
0,54.0,0,0,1,0
1,54.0,1,0,0,0
2,54.0,0,0,0,1
3,54.0,0,1,0,0
4,53.0,0,0,1,0


In [13]:
#get shape of target array
y.shape

(7091, 1)

In [14]:
#get shape of feautures array
X.shape

(7091, 5)

In [15]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [16]:
#create linear regression model using sklearn
model = LinearRegression()

In [17]:
#fit the model with training data
regression_age_MUA_mortality = model.fit(X_train, y_train)

In [18]:
#Create predictions array using linear regression
y_pred = regression_age_MUA_mortality.predict(X_test)
y_pred

array([[559.37872162],
       [ 58.153283  ],
       [926.7694535 ],
       ...,
       [559.24589687],
       [559.54949629],
       [559.54949629]])

In [19]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9435616251834894

In [20]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [21]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [22]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                 2.848e+04
Date:                Sun, 29 May 2022   Prob (F-statistic):               0.00
Time:                        20:54:23   Log-Likelihood:                -42174.
No. Observations:                7091   AIC:                         8.436e+04
Df Residuals:                    7086   BIC:                         8.439e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [23]:
#fit the model with training data
#decision tree regression
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [24]:
#predict
y_pred = regressor_DT.predict(X_test)
y_pred

array([568.12542373,  57.33535354, 940.18108108, ..., 592.63076923,
       563.43636364, 563.43636364])

In [25]:
#calculate r2 score
r2_score(y_test, y_pred)

0.9418049398853228